In [ ]:
import os
import time
import requests
import pandas as pd
import mysql.connector
from datetime import datetime

# Initialize API parameters
url = "https://api.coingecko.com/api/v3/coins/markets"
pages = 4
per_page = 250

# Fetch data from API
total_data = []
for page in range(1, pages + 1):
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": per_page,
        "page": page,
        "sparkline": "false",
        "price_change_percentage": "7d,30d,1y"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        total_data.extend(response.json())
    else:
        print(f"Error obtaining page {page}: {response.status_code}")
    time.sleep(1)  # Avoid API rate limits

print(f"Total coins fetched: {len(total_data)}")

# Convert to DataFrame
df = pd.DataFrame(total_data)
df["fetch_date"] = pd.Timestamp.now()

# Select relevant columns
df = df[[
    "id", "symbol", "name", "image", "current_price", "market_cap",
    "market_cap_rank", "total_volume", "high_24h", "low_24h",
    "price_change_24h", "price_change_percentage_24h",
    "price_change_percentage_7d_in_currency",
    "price_change_percentage_30d_in_currency",
    "price_change_percentage_1y_in_currency",
    "market_cap_change_24h", "total_supply", "max_supply",
    "ath", "atl", "ath_date", "atl_date", "fetch_date"
]]

# Clean up column names for MySQL
df.columns = [col.upper().replace(" ", "_") for col in df.columns]

# Fix datetime formats
for col in ["ATH_DATE", "ATL_DATE", "FETCH_DATE"]:
    df[col] = pd.to_datetime(df[col], errors="coerce")
    df[col] = df[col].dt.strftime("%Y-%m-%d %H:%M:%S")

# Connect to MySQL
print("Connecting to MySQL...")
try:
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Alexandersql7?",
        database="crypto_db"
    )
    cursor = conn.cursor()

    # 🔧 Create / update table structure
    create_table_query = """
    CREATE TABLE IF NOT EXISTS crypto_market (
        ID VARCHAR(255),
        SYMBOL VARCHAR(50),
        NAME VARCHAR(255),
        IMAGE_URL VARCHAR(500),
        CURRENT_PRICE DOUBLE,
        MARKET_CAP BIGINT,
        MARKET_CAP_RANK INT,
        TOTAL_VOLUME BIGINT,
        HIGH_24H DOUBLE,
        LOW_24H DOUBLE,
        PRICE_CHANGE_24H DOUBLE,
        PRICE_CHANGE_PERCENTAGE_24H DOUBLE,
        PRICE_CHANGE_7D DOUBLE,
        PRICE_CHANGE_30D DOUBLE,
        PRICE_CHANGE_1Y DOUBLE,
        MARKET_CAP_CHANGE_24H DOUBLE,
        TOTAL_SUPPLY DOUBLE,
        MAX_SUPPLY DOUBLE,
        ATH DOUBLE,
        ATL DOUBLE,
        ATH_DATE DATETIME,
        ATL_DATE DATETIME,
        FETCH_DATE DATETIME
    )
    """
    cursor.execute(create_table_query)
    conn.commit()

    # Insert rows
    insert_query = """
    INSERT INTO crypto_market (
        ID, SYMBOL, NAME, IMAGE_URL, CURRENT_PRICE, MARKET_CAP, MARKET_CAP_RANK, TOTAL_VOLUME,
        HIGH_24H, LOW_24H, PRICE_CHANGE_24H, PRICE_CHANGE_PERCENTAGE_24H,
        PRICE_CHANGE_7D, PRICE_CHANGE_30D, PRICE_CHANGE_1Y, MARKET_CAP_CHANGE_24H,
        TOTAL_SUPPLY, MAX_SUPPLY, ATH, ATL, ATH_DATE, ATL_DATE, FETCH_DATE
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    data_tuples = [tuple(None if pd.isna(x) else x for x in row) for row in df.values]
    cursor.executemany(insert_query, data_tuples)
    conn.commit()

    print(f"Successfully inserted {cursor.rowcount} rows into MySQL.")

except mysql.connector.Error as err:
    print("MySQL Error:", err)
except Exception as e:
    print("Unexpected Error:", e)
finally:
    if 'cursor' in locals() and cursor:
        cursor.close()
    if 'conn' in locals() and conn.is_connected():
        conn.close()

# Preview the data
print(df.head())

Total coins fetched: 1000
Connecting to MySQL...
Successfully inserted 1000 rows into MySQL.
            ID SYMBOL      NAME  \
0      bitcoin    btc   Bitcoin   
1     ethereum    eth  Ethereum   
2       tether   usdt    Tether   
3  binancecoin    bnb       BNB   
4       ripple    xrp       XRP   

                                               IMAGE  CURRENT_PRICE  \
0  https://coin-images.coingecko.com/coins/images...     110991.000   
1  https://coin-images.coingecko.com/coins/images...       3985.810   
2  https://coin-images.coingecko.com/coins/images...          1.001   
3  https://coin-images.coingecko.com/coins/images...       1163.320   
4  https://coin-images.coingecko.com/coins/images...          2.410   

      MARKET_CAP  MARKET_CAP_RANK  TOTAL_VOLUME    HIGH_24H     LOW_24H  ...  \
0  2212881079023                1  6.916429e+10  113527.000  110392.000  ...   
1   480995591993                2  4.499161e+10    4203.800    3944.170  ...   
2   180754050872             